<a href="https://colab.research.google.com/github/inshra12/ESM-MLP-baseline-protein-function/blob/main/notebooks/sequence-based-function-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from data_loader import load_data,encode_labels

In [2]:
!pip install fair-esm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 751.8 kB/s eta 0:00:00


In [3]:
from esm_embedder import save_embeddings

In [4]:
df=load_data("/content/raw_sequence_data.csv")

First protein entry:
 ID                                                     P14416
Sequence    MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...
MF          GO:0035240;GO:0001591;GO:0004930; GO:0001965;G...
CC          GO:0001669; GO:0030424; GO:0043679; GO:0060170...
BP          GO:0046717; GO:0021984; GO:0007195; GO:0007628...
Labels      [ GO:0045745,  GO:0043679, GO:0046717,  GO:009...
Name: 0, dtype: object


In [5]:
train_df, test_df = split_data(df)

In [6]:
# Encode the training labels
y_train, mlb = encode_labels(train_df['Labels'])

In [7]:
# Encode test labels (important: use the same `mlb`)
y_test= mlb.transform(test_df['Labels'])

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [' GO:0000139', ' GO:0001540', ' GO:0001755', ' GO:0001819', ' GO:0001938', ' GO:0001976', ' GO:0002024', ' GO:0002025', ' GO:0002027', ' GO:0002028', ' GO:0002052', ' GO:0002092', ' GO:0003007', ' GO:0003063', ' GO:0003300', ' GO:0004941', ' GO:0005096', ' GO:0005634', ' GO:0005789', ' GO:0005794', ' GO:0005929', ' GO:0006366', ' GO:0006914', ' GO:0006939', ' GO:0006940', ' GO:0007207', ' GO:0007270', ' GO:0007405', ' GO:0007409', ' GO:0007416', ' GO:0007608', ' GO:0007616', ' GO:0007625', ' GO:0007628', ' GO:0008104', ' GO:0008306', ' GO:0009409', ' GO:0009416', ' GO:0009636', ' GO:0009925', ' GO:0010008', ' GO:0010039', ' GO:0014033', ' GO:0014827', ' GO:0014854', ' GO:0015459', ' GO:0015630', ' GO:0016055', ' GO:0016323', ' GO:0016328', ' GO:0016907', ' GO:0019934', ' GO:0021756', ' GO:0021769', ' GO:0021853', ' GO:0021984', ' GO:0030139', ' GO:0030336', ' GO:0030424', ' GO:0

In [9]:
import pandas as pd
# Convert encoded labels to DataFrames with proper column names
train_labels_df = pd.DataFrame(y_train, columns=mlb.classes_)
train_labels_df.insert(0, "ID", train_df["ID"])  # Insert IDs as first column

test_labels_df = pd.DataFrame(y_test, columns=mlb.classes_)
test_labels_df.insert(0, "ID", test_df["ID"])


In [12]:
# Save to CSV
train_labels_df.to_csv("train_encoded_labels.csv", index=False)
test_labels_df.to_csv("test_encoded_labels.csv", index=False)

print("✅ Saved encoded train and test labels to CSV.")

✅ Saved encoded train and test labels to CSV.


In [ ]:

# Convert the binary matrix to a DataFrame for readability
#encoded_df = pd.DataFrame(y_train, columns=mlb.classes_)

# Show the first few rows
#print(encoded_df.head())

In [ ]:
#save_embeddings("/content/raw_sequence_data.csv", "cls_embeddings.csv")